In [44]:
import matplotlib.pylab as plt, pandas as pd, MySQLdb
from matplotlib import font_manager
myFont = font_manager.FontProperties(fname='/Library/Fonts/Songti.ttc')
titleSize = 12
tipSize = 10

In [2]:
conn = MySQLdb.connect(host='localhost',user='root',passwd='',db='qyw', charset='utf8')

In [147]:
df = pd.read_sql('''
    SELECT PROVINCE, CITY, COUNT(DISTINCT USER_ID) AS USER_CNT FROM (SELECT * FROM (SELECT * FROM qyw.qyw_4th_visit WHERE USER_ID > 0 AND HOSPITAL_ID = 270001 ORDER BY USER_ID) AS t1 INNER JOIN (SELECT USER_ID AS CUS_ID, PROVINCE, CITY, BIRTHDAY, GENDER, REGISTER_DATE FROM qyw.qyw_4th_user ORDER BY CUS_ID) AS t2 ON t1.USER_ID = t2.CUS_ID) AS t3 WHERE PROVINCE IS NOT null AND CITY IS NOT null GROUP BY PROVINCE, CITY ORDER BY USER_CNT DESC;
    ''', con=conn)

In [4]:
province_city_cnt = pd.Series(df['USER_CNT'].values, index=df['PROVINCE']+' '+df['CITY'], name='用户地域分布')

In [74]:
fig = plt.figure(dpi=100, figsize=(8,8))
# first axes
ax1 = fig.add_axes([0.1, 0.1, 0.8, 0.8])
patches, texts, autotexts = ax1.pie(province_city_cnt[1:11], labels=province_city_cnt[1:11].index, autopct='%1.1f%%', colors=['yellowgreen', 'gold', 'lightskyblue', 'lightcoral'])
plt.setp(autotexts, fontproperties=myFont, size=tipSize)
plt.setp(texts, fontproperties=myFont, size=tipSize)
ax1.set_title(u'用户地域分布（第二位至第十一位）',fontproperties=myFont, size=titleSize)
ax1.set_aspect(1)
plt.show()

In [148]:
df_all = pd.read_sql('''
SELECT HOSPITAL_ID, PROVINCE, CITY, COUNT(DISTINCT USER_ID) AS USER_CNT FROM (SELECT * FROM (SELECT * FROM qyw.qyw_4th_visit WHERE USER_ID > 0 ORDER BY USER_ID) AS t1 INNER JOIN (SELECT USER_ID AS CUS_ID, PROVINCE, CITY, BIRTHDAY, GENDER, REGISTER_DATE FROM qyw.qyw_4th_user ORDER BY CUS_ID) AS t2 ON t1.USER_ID = t2.CUS_ID) AS t3 WHERE PROVINCE IS NOT null AND CITY IS NOT null GROUP BY HOSPITAL_ID, PROVINCE, CITY ORDER BY HOSPITAL_ID, USER_CNT DESC;
''', con=conn)

In [149]:
pd.DataFrame(df_all.groupby(['PROVINCE', 'CITY'])['USER_CNT'].sum()).reset_index().sort_values(['USER_CNT'],0,[False])

,PROVINCE,CITY,USER_CNT
86,湖北,武汉,5362
12,安徽,合肥,2345
25,安徽,阜阳,101
11,安徽,六安,85
20,安徽,淮南,62
13,安徽,安庆,44
0,上海,上海,44
39,广东,广州,42
90,湖北,襄樊,32
22,安徽,芜湖,32


In [187]:
df_new = df_all.groupby(['HOSPITAL_ID','PROVINCE'])['USER_CNT'].sum().reset_index().sort_values(['PROVINCE','HOSPITAL_ID'],0,[True, False])
df_new = pd.merge(df_new[df_new['PROVINCE']!=u'安徽'], df_new[df_new['PROVINCE']!=u'湖北'])
# useless below
# array_province = df_new['PROVINCE'].drop_duplicates().values
# hospitals = [{270001:u'武汉市中心医院'}, {5510002:u'安徽省中医院'}]
# hos_cnts = {270001:[], 5510002:[]}
# for hospital in hospitals:
#     for hid, hname in hospital.items():
#         for province in array_province:
#             if df_new[df_new['HOSPITAL_ID']==hid]['PROVINCE'] == None:
#                 hos_cnts[hid].append(0)
#             else:
#                 hos_cnts[hid].append(df_new[df_new['HOSPITAL_ID']==hid]['PROVINCE'])
# print hos_cnts
# useless above
df_new_pivot = df_new.pivot('PROVINCE', 'HOSPITAL_ID', 'USER_CNT') # equal to above
df_new_pivot = df_new_pivot.fillna(0)

import numpy as np
import matplotlib.pyplot as plt


n_groups = df_new_pivot.index.size

hid_270001 = df_new_pivot[270001]

hid_5510002 = df_new_pivot[5510002]

fig, ax = plt.subplots(dpi=100, figsize=(14,8))

index = np.arange(n_groups)
bar_width = 0.35

opacity = 0.4
error_config = {'ecolor': '0.3'}

rects1 = plt.bar(index, hid_270001, bar_width,
                 alpha=opacity,
                 color='b',
                 error_kw=error_config,
                 label=u'武汉市中心医院')
    
rects2 = plt.bar(index + bar_width, hid_5510002, bar_width,
                 alpha=opacity,
                 color='r',
                 error_kw=error_config,
                 label=u'安徽省中医院')

plt.xlabel(u'省份', fontproperties=myfont, size=titleSize)
plt.ylabel(u'用户数量', fontproperties=myfont, size=titleSize)
plt.title(u'用户地域分布', fontproperties=myfont, size=titleSize)
plt.xticks(index + bar_width, df_new_pivot.index, fontproperties=myfont, size=tipSize)
plt.legend(prop=font_manager.FontProperties(fname='/Library/Fonts/Songti.ttc', size=tipSize))

def autolabel(rects):
    # attach some text labels
    for rect in rects:
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., 1.005*height,
                '%d' % int(height),
                ha='center', va='bottom', fontproperties=myfont, size=tipSize)

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()

In [ ]:
conn.close()